This file will download the data of BMT cohorts from roadmap. Following are the titles and their ID in the admin portal:
•	201 = Caregivers- Roadmap 2.0 (U-M, intervention group)
•	202 = Patients- Roadmap 2.0
•	203 = Control Caregivers-Roadmap 2.0 (U-M)
•	321 = OHSU - BMT Control Caregivers
•	322 = OHSU - BMT Patients
•	323 = OHSU - BMT PA Caregivers


In [1]:
import cx_Oracle
# import sqlalchemy
import numpy as np
import getpass
import oracledb
import cx_Oracle
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import os
import csv
import datetime
import pyodbc

In [2]:
# Enfore the DSN String has the service_name field instead of the SID
dsnStr = cx_Oracle.makedsn("rmap.mcit.med.umich.edu", "1521", service_name="RMAP.WORLD")
connection = oracledb.connect(
    user="rajnishk",
    password="DU8RN7WE",
    dsn=dsnStr
)

print("Successfully connected to Oracle Database")

Successfully connected to Oracle Database


In [3]:
cursor = connection.cursor()

In [4]:
Caregivers_Roadmap_2__0_U__M_intervention_group = 201
Patients_Roadmap_2__0 = 202
Control_Caregivers__Roadmap_2__0_U_M = 203
OHSU_BMT_Control_Caregivers = 321
OHSU_BMT_Patients = 322
OHSU_BMT_PA_Caregivers = 323

In [5]:
list_of_studies = [201,202,203,321,322,323]

In [6]:
cursor.execute(
    "select *from ROADMAP.PRTCPT_DVC where STUDY_ID = '" + str(Caregivers_Roadmap_2__0_U__M_intervention_group) + "'"
)

unique_strings = cursor.fetchall()
column_names   = [i[0] for i in cursor.description]
df_test = pd.DataFrame(unique_strings,columns=column_names)
df_test

,PRTCPT_DVC_ID,PRTCPT_DVC_TYP_ID,STUDY_PRTCPT_ID,STUDY_ID,PRTCPT_DVC_MAKE,PRTCPT_DVC_MODEL,PRTCPT_DVC_INTERNAL_ID,PRTCPT_DVC_START_DT,PRTCPT_DVC_END_DT,FITBIT_SYNC_TIME
0,1147,2,PUFGNCG6,201,iOS,iPhone,F38EB8B3-09E5-4FDA-9261-11DF9F85F7AD,2020-08-19 10:44:56.213,2021-02-22 12:43:49.836,NaT
1,1098,2,JZHKYTVN,201,iOS,iPhone,6DE3402C-85C8-491F-A05C-B7BA64E9B344,2020-07-24 10:01:32.081,NaT,NaT
2,1320,2,V3E5XTNB,201,iOS,iPhone,4D5F628B-A0CE-4B28-A013-D98ABB00E8EF,2020-09-15 12:59:28.546,2021-02-24 18:04:31.218,NaT
3,1332,1,V3E5XTNB,201,Fitbit,Charge HR,8V5VJT,2020-09-16 13:19:27.053,2020-09-23 11:02:11.196,2020-09-23 10:36:26
4,1744,2,ZLZ19K6V,201,iOS,iPhone,7696BC93-BBF4-40E2-8C80-9BE10373355C,2020-10-01 10:04:03.501,2020-11-11 15:50:09.558,NaT
...,...,...,...,...,...,...,...,...,...,...
246,27753,1,YFTMNY6X,201,Fitbit,Charge HR,BHDJSX,2023-04-10 11:57:47.375,NaT,2023-10-06 08:21:41
247,27813,2,7QK5PT5A,201,iOS,iPhone,7DD4C10D-3E93-4FDF-BC67-DD1DD94C5DB5,2023-05-01 10:50:41.261,2023-06-07 14:15:54.373,NaT
248,27833,1,7QK5PT5A,201,Fitbit,Charge HR,23QB25,2023-05-01 11:31:20.417,NaT,2023-10-06 08:38:32
249,27853,2,Y8K512GQ,201,iOS,iPhone,51C1FD3B-0309-4809-9A6E-13FC5249EE7D,2023-05-02 12:03:30.146,2023-09-29 14:01:35.847,NaT


In [7]:
len(df_test.STUDY_PRTCPT_ID.unique())

90

In [8]:
cursor.execute(
    "select *from ROADMAP.PRTCPT_DVC where STUDY_ID = '" + str(list_of_studies[0]) + "'"
)

unique_strings = cursor.fetchall()
column_names   = [i[0] for i in cursor.description]
PRTCPTS_BMT = pd.DataFrame(unique_strings,columns=column_names)
PRTCPTS_BMT

,PRTCPT_DVC_ID,PRTCPT_DVC_TYP_ID,STUDY_PRTCPT_ID,STUDY_ID,PRTCPT_DVC_MAKE,PRTCPT_DVC_MODEL,PRTCPT_DVC_INTERNAL_ID,PRTCPT_DVC_START_DT,PRTCPT_DVC_END_DT,FITBIT_SYNC_TIME
0,1147,2,PUFGNCG6,201,iOS,iPhone,F38EB8B3-09E5-4FDA-9261-11DF9F85F7AD,2020-08-19 10:44:56.213,2021-02-22 12:43:49.836,NaT
1,1098,2,JZHKYTVN,201,iOS,iPhone,6DE3402C-85C8-491F-A05C-B7BA64E9B344,2020-07-24 10:01:32.081,NaT,NaT
2,1320,2,V3E5XTNB,201,iOS,iPhone,4D5F628B-A0CE-4B28-A013-D98ABB00E8EF,2020-09-15 12:59:28.546,2021-02-24 18:04:31.218,NaT
3,1332,1,V3E5XTNB,201,Fitbit,Charge HR,8V5VJT,2020-09-16 13:19:27.053,2020-09-23 11:02:11.196,2020-09-23 10:36:26
4,1744,2,ZLZ19K6V,201,iOS,iPhone,7696BC93-BBF4-40E2-8C80-9BE10373355C,2020-10-01 10:04:03.501,2020-11-11 15:50:09.558,NaT
...,...,...,...,...,...,...,...,...,...,...
246,27753,1,YFTMNY6X,201,Fitbit,Charge HR,BHDJSX,2023-04-10 11:57:47.375,NaT,2023-10-06 08:21:41
247,27813,2,7QK5PT5A,201,iOS,iPhone,7DD4C10D-3E93-4FDF-BC67-DD1DD94C5DB5,2023-05-01 10:50:41.261,2023-06-07 14:15:54.373,NaT
248,27833,1,7QK5PT5A,201,Fitbit,Charge HR,23QB25,2023-05-01 11:31:20.417,NaT,2023-10-06 08:38:32
249,27853,2,Y8K512GQ,201,iOS,iPhone,51C1FD3B-0309-4809-9A6E-13FC5249EE7D,2023-05-02 12:03:30.146,2023-09-29 14:01:35.847,NaT


In [9]:
for i in range(1, len(list_of_studies)):
    print("current study id being downloaded is", list_of_studies[i])
    cursor.execute(
        "select *from ROADMAP.PRTCPT_DVC where STUDY_ID = '" + str(list_of_studies[i]) + "'"
    )

    unique_strings = cursor.fetchall()
    column_names   = [i[0] for i in cursor.description]
    df_current = pd.DataFrame(unique_strings,columns=column_names)
    PRTCPTS_BMT = pd.concat([PRTCPTS_BMT,df_current],ignore_index= True)
    print("cohort ID __ is done", list_of_studies[i])
    


current study id being downloaded is 202
cohort ID __ is done 202
current study id being downloaded is 203
cohort ID __ is done 203
current study id being downloaded is 321
cohort ID __ is done 321
current study id being downloaded is 322
cohort ID __ is done 322
current study id being downloaded is 323
cohort ID __ is done 323


In [10]:
PRTCPTS_BMT

,PRTCPT_DVC_ID,PRTCPT_DVC_TYP_ID,STUDY_PRTCPT_ID,STUDY_ID,PRTCPT_DVC_MAKE,PRTCPT_DVC_MODEL,PRTCPT_DVC_INTERNAL_ID,PRTCPT_DVC_START_DT,PRTCPT_DVC_END_DT,FITBIT_SYNC_TIME
0,1147,2,PUFGNCG6,201,iOS,iPhone,F38EB8B3-09E5-4FDA-9261-11DF9F85F7AD,2020-08-19 10:44:56.213,2021-02-22 12:43:49.836,NaT
1,1098,2,JZHKYTVN,201,iOS,iPhone,6DE3402C-85C8-491F-A05C-B7BA64E9B344,2020-07-24 10:01:32.081,NaT,NaT
2,1320,2,V3E5XTNB,201,iOS,iPhone,4D5F628B-A0CE-4B28-A013-D98ABB00E8EF,2020-09-15 12:59:28.546,2021-02-24 18:04:31.218,NaT
3,1332,1,V3E5XTNB,201,Fitbit,Charge HR,8V5VJT,2020-09-16 13:19:27.053,2020-09-23 11:02:11.196,2020-09-23 10:36:26
4,1744,2,ZLZ19K6V,201,iOS,iPhone,7696BC93-BBF4-40E2-8C80-9BE10373355C,2020-10-01 10:04:03.501,2020-11-11 15:50:09.558,NaT
...,...,...,...,...,...,...,...,...,...,...
977,17594,1,8GUM4J6X,323,Fitbit,Charge HR,9NGF82,2021-11-09 14:45:37.950,2022-03-12 07:17:34.828,2022-03-12 06:15:06
978,19973,2,TQR5TQ7Y,323,android,samsung SM-G781U,1643754487847,2022-02-01 17:28:09.259,2022-07-13 12:58:20.812,NaT
979,19993,2,1FUAH98J,323,android,samsung SM-G781U,1643754885693,2022-02-01 17:34:46.532,2022-07-12 16:59:27.333,NaT
980,20133,2,3BELGA8R,323,iOS,iPhone,E8140A75-2232-4816-8F20-3B5E1C18A4ED,2022-02-09 12:13:58.829,2022-07-13 12:56:38.246,NaT


In [22]:
PRTCPTS_BMT[PRTCPTS_BMT.STUDY_ID==203]

,PRTCPT_DVC_ID,PRTCPT_DVC_TYP_ID,STUDY_PRTCPT_ID,STUDY_ID,PRTCPT_DVC_MAKE,PRTCPT_DVC_MODEL,PRTCPT_DVC_INTERNAL_ID,PRTCPT_DVC_START_DT,PRTCPT_DVC_END_DT,FITBIT_SYNC_TIME
725,1397,2,ZFZV5EQD,203,iOS,iPhone,CE577822-544A-43DA-A164-81C9DC9914F0,2020-09-22 14:40:49.516,2021-01-18 12:40:54.853,NaT
726,1593,2,LY6EY19F,203,android,Google Pixel 2 XL,1601388150941,2020-09-29 10:02:31.095,2020-12-11 18:29:27.190,NaT
727,1725,1,LY6EY19F,203,Fitbit,Charge HR,2C6MN5,2020-09-30 12:58:27.866,NaT,2022-06-05 02:19:15
728,4215,1,IDFFG6QW,203,Fitbit,Charge HR,8WVX88,2020-10-23 19:01:24.892,NaT,2023-10-18 13:15:57
729,4217,1,ZFZV5EQD,203,Fitbit,Charge HR,8WSLQG,2020-10-23 19:03:12.100,NaT,2023-10-06 17:36:14
...,...,...,...,...,...,...,...,...,...,...
928,25176,1,6TTX519N,203,Fitbit,Charge HR,B8WF3L,2022-11-17 15:22:08.976,NaT,2023-10-07 04:57:56
929,25753,1,7R3FPVPM,203,Fitbit,Charge HR,46HZCX,2022-12-16 15:25:58.982,NaT,2023-10-18 13:15:08
930,26174,1,6TG7E6IB,203,Fitbit,Charge HR,58372D,2023-01-20 14:50:07.032,NaT,2023-10-07 08:02:24
931,27655,1,L9DED7AU,203,Fitbit,Charge HR,BHZHFB,2023-04-04 16:56:15.746,NaT,2023-10-18 13:15:19


In [20]:
PRTCPTS_BMT[PRTCPTS_BMT.STUDY_ID==202 & PRTCPTS_BMT.PRTCPT_DVC_TYP_ID==1]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [14]:
PRTCPTS_BMT.STUDY_ID.value_counts()

STUDY_ID
202    474
201    251
203    208
322     24
323     18
321      7
Name: count, dtype: int64

In [17]:
len(PRTCPTS_BMT.STUDY_PRTCPT_ID.unique())

396

In [18]:
PRTCPTS_BMT_fitbit_rows = PRTCPTS_BMT[PRTCPTS_BMT.PRTCPT_DVC_MAKE =="Fitbit"]
PRTCPTS_BMT_fitbit_rows

,PRTCPT_DVC_ID,PRTCPT_DVC_TYP_ID,STUDY_PRTCPT_ID,STUDY_ID,PRTCPT_DVC_MAKE,PRTCPT_DVC_MODEL,PRTCPT_DVC_INTERNAL_ID,PRTCPT_DVC_START_DT,PRTCPT_DVC_END_DT,FITBIT_SYNC_TIME
3,1332,1,V3E5XTNB,201,Fitbit,Charge HR,8V5VJT,2020-09-16 13:19:27.053,2020-09-23 11:02:11.196,2020-09-23 10:36:26
5,1717,1,DCPPCCLC,201,Fitbit,Charge HR,8VXYTW,2020-09-30 11:16:03.404,2021-02-01 12:48:09.228,2021-02-01 12:15:03
7,1418,1,V3E5XTNB,201,Fitbit,Charge HR,8V5VJT,2020-09-23 11:02:29.784,NaT,2023-10-06 16:47:18
9,1261,1,7L778454,201,Fitbit,Charge HR,3B9WW3,2020-09-08 22:22:52.729,NaT,2022-06-05 00:33:24
11,1364,1,VN96JBSP,201,Fitbit,Charge HR,8TZFFC,2020-09-18 11:09:40.938,NaT,2023-10-07 06:22:09
...,...,...,...,...,...,...,...,...,...,...
972,19853,1,3VEZSKKS,323,Fitbit,Charge HR,3SF7ZW,2022-01-28 13:40:29.659,NaT,2023-10-07 07:58:32
973,20213,1,3BELGA8R,323,Fitbit,Charge HR,9VSPXK,2022-02-09 19:11:19.265,NaT,2023-10-07 09:05:36
974,19674,1,CEFKIZ47,323,Fitbit,Charge HR,9TLWSL,2022-01-21 18:40:38.176,NaT,2022-08-25 22:59:41
976,20233,1,1FUAH98J,323,Fitbit,Charge HR,267TB2,2022-02-09 22:24:51.065,NaT,2023-10-06 11:24:55


Looks like 52 people have multiple fitbits (PRTCPT_DVC_ID) on them.  

In [19]:
len(PRTCPTS_BMT_fitbit_rows.STUDY_PRTCPT_ID.unique())

356

In [20]:
# PRTCPTS_BMT_fitbit_rows.STUDY_PRTCPT_ID.value_counts()[0:52] 
len(PRTCPTS_BMT_fitbit_rows.PRTCPT_DVC_ID.unique())


551

Let us get all the participant device IDs. Several of them are repeated, but for now I will just download the heartrate data. 

In [21]:
BMT_PRTCPT_DVC_ID = PRTCPTS_BMT_fitbit_rows.PRTCPT_DVC_ID.unique()
len(BMT_PRTCPT_DVC_ID)

551

In [17]:
# df_heartrate_BMT_PRTCPT_DVC_ID = pd.DataFrame(columns=['STUDY_METRIC_ID',
# 'STUDY_METRIC_TYP_ID',
# 'PRTCPT_DVC_ID',
# 'PRTCPT_DVC_TYP_ID',
# 'STUDY_METRIC_PRTCPTN_IND',
# 'STUDY_METRIC_MSR_VAL',
# 'STUDY_METRIC_MSR_START_DT',
# 'STUDY_METRIC_MSR_END_DT'])

In [18]:
# for i in range(len(BMT_PRTCPT_DVC_ID)):
#     print("downloading heartrate dataset for device number",i)
#     cursor.execute(
#     "select  * from roadmap.STUDY_METRIC_HEARTRATE where PRTCPT_DVC_ID = '" + str(BMT_PRTCPT_DVC_ID[i]) + "'"
#     )
#     unique_strings = cursor.fetchall()
#     column_names   = [i[0] for i in cursor.description]
#     df_current    = pd.DataFrame(unique_strings,columns=column_names)
#     df_heartrate_BMT_PRTCPT_DVC_ID  = pd.concat([df_heartrate_BMT_PRTCPT_DVC_ID,df_current],ignore_index=True)    
    

downloading heartrate dataset for device number 0
downloading heartrate dataset for device number 1
downloading heartrate dataset for device number 2
downloading heartrate dataset for device number 3
downloading heartrate dataset for device number 4
downloading heartrate dataset for device number 5
downloading heartrate dataset for device number 6
downloading heartrate dataset for device number 7
downloading heartrate dataset for device number 8
downloading heartrate dataset for device number 9
downloading heartrate dataset for device number 10
downloading heartrate dataset for device number 11
downloading heartrate dataset for device number 12
downloading heartrate dataset for device number 13
downloading heartrate dataset for device number 14
downloading heartrate dataset for device number 15
downloading heartrate dataset for device number 16
downloading heartrate dataset for device number 17
downloading heartrate dataset for device number 18
downloading heartrate dataset for device 

downloading heartrate dataset for device number 160
downloading heartrate dataset for device number 161
downloading heartrate dataset for device number 162
downloading heartrate dataset for device number 163
downloading heartrate dataset for device number 164
downloading heartrate dataset for device number 165
downloading heartrate dataset for device number 166
downloading heartrate dataset for device number 167
downloading heartrate dataset for device number 168
downloading heartrate dataset for device number 169
downloading heartrate dataset for device number 170
downloading heartrate dataset for device number 171
downloading heartrate dataset for device number 172
downloading heartrate dataset for device number 173
downloading heartrate dataset for device number 174
downloading heartrate dataset for device number 175
downloading heartrate dataset for device number 176
downloading heartrate dataset for device number 177
downloading heartrate dataset for device number 178
downloading 

downloading heartrate dataset for device number 318
downloading heartrate dataset for device number 319
downloading heartrate dataset for device number 320
downloading heartrate dataset for device number 321
downloading heartrate dataset for device number 322
downloading heartrate dataset for device number 323
downloading heartrate dataset for device number 324
downloading heartrate dataset for device number 325
downloading heartrate dataset for device number 326
downloading heartrate dataset for device number 327
downloading heartrate dataset for device number 328
downloading heartrate dataset for device number 329
downloading heartrate dataset for device number 330
downloading heartrate dataset for device number 331
downloading heartrate dataset for device number 332
downloading heartrate dataset for device number 333
downloading heartrate dataset for device number 334
downloading heartrate dataset for device number 335
downloading heartrate dataset for device number 336
downloading 

downloading heartrate dataset for device number 476
downloading heartrate dataset for device number 477
downloading heartrate dataset for device number 478
downloading heartrate dataset for device number 479
downloading heartrate dataset for device number 480
downloading heartrate dataset for device number 481
downloading heartrate dataset for device number 482
downloading heartrate dataset for device number 483
downloading heartrate dataset for device number 484
downloading heartrate dataset for device number 485
downloading heartrate dataset for device number 486
downloading heartrate dataset for device number 487
downloading heartrate dataset for device number 488
downloading heartrate dataset for device number 489
downloading heartrate dataset for device number 490
downloading heartrate dataset for device number 491
downloading heartrate dataset for device number 492
downloading heartrate dataset for device number 493
downloading heartrate dataset for device number 494
downloading 

In [19]:
# df_heartrate_BMT_PRTCPT_DVC_ID

,STUDY_METRIC_ID,STUDY_METRIC_TYP_ID,PRTCPT_DVC_ID,PRTCPT_DVC_TYP_ID,STUDY_METRIC_PRTCPTN_IND,STUDY_METRIC_MSR_VAL,STUDY_METRIC_MSR_START_DT,STUDY_METRIC_MSR_END_DT
0,42625863,3,1332,1,None,70,2020-09-16 12:33:28,2020-09-16 12:34:28
1,42625864,3,1332,1,None,71,2020-09-16 12:33:38,2020-09-16 12:34:38
2,42625865,3,1332,1,None,80,2020-09-16 12:33:43,2020-09-16 12:34:43
3,42625866,3,1332,1,None,81,2020-09-16 12:33:48,2020-09-16 12:34:48
4,42625867,3,1332,1,None,83,2020-09-16 12:33:53,2020-09-16 12:34:53
...,...,...,...,...,...,...,...,...
85648977,824555768,3,17594,1,None,48,2022-03-11 10:43:00,2022-03-11 10:44:00
85648978,824555769,3,17594,1,None,46,2022-03-11 10:44:00,2022-03-11 10:45:00
85648979,824555770,3,17594,1,None,48,2022-03-11 10:45:00,2022-03-11 10:46:00
85648980,824555771,3,17594,1,None,50,2022-03-11 10:46:00,2022-03-11 10:47:00


In [21]:
# df_heartrate_BMT_PRTCPT_DVC_ID.to_csv('BMT_heartrate.csv')

In [22]:
# df_heartrate_BMT_PRTCPT_DVC_ID

In [23]:
!dir

 Volume in drive C is OS-APPS
 Volume Serial Number is A6DB-57A1

 Directory of C:\Users\rajnishk\OneDrive - Michigan Medicine\Documents\Python Scripts\K24 grant

10/18/2023  09:22 AM    <DIR>          .
10/18/2023  09:22 AM    <DIR>          ..
10/16/2023  03:36 PM    <DIR>          .ipynb_checkpoints
10/18/2023  09:22 AM           187,073 BMT data heartrate.ipynb
10/03/2023  04:56 PM     6,347,199,374 BMT_heartrate.csv
10/09/2023  11:56 AM           167,114 BMT_heartrate.ipynb
10/05/2023  12:43 PM            32,777 BMT_Moods.ipynb
10/03/2023  04:51 PM        11,540,152 BMT_sleep.csv
10/03/2023  05:12 PM            54,143 BMT_sleep.ipynb
10/09/2023  06:40 PM            52,254 BMT_steps.ipynb
10/16/2023  05:16 PM            50,303 Preliminary_analysis_hr.ipynb
               8 File(s)  6,359,283,190 bytes
               3 Dir(s)  331,411,800,064 bytes free


In [2]:
df_HR = pd.read_csv('BMT_heartrate.csv')
df_HR

,Unnamed: 0,STUDY_METRIC_ID,STUDY_METRIC_TYP_ID,PRTCPT_DVC_ID,PRTCPT_DVC_TYP_ID,STUDY_METRIC_PRTCPTN_IND,STUDY_METRIC_MSR_VAL,STUDY_METRIC_MSR_START_DT,STUDY_METRIC_MSR_END_DT
0,0,42625863,3,1332,1,NaN,70,2020-09-16 12:33:28,2020-09-16 12:34:28
1,1,42625864,3,1332,1,NaN,71,2020-09-16 12:33:38,2020-09-16 12:34:38
2,2,42625865,3,1332,1,NaN,80,2020-09-16 12:33:43,2020-09-16 12:34:43
3,3,42625866,3,1332,1,NaN,81,2020-09-16 12:33:48,2020-09-16 12:34:48
4,4,42625867,3,1332,1,NaN,83,2020-09-16 12:33:53,2020-09-16 12:34:53
...,...,...,...,...,...,...,...,...,...
85648977,85648977,824555768,3,17594,1,NaN,48,2022-03-11 10:43:00,2022-03-11 10:44:00
85648978,85648978,824555769,3,17594,1,NaN,46,2022-03-11 10:44:00,2022-03-11 10:45:00
85648979,85648979,824555770,3,17594,1,NaN,48,2022-03-11 10:45:00,2022-03-11 10:46:00
85648980,85648980,824555771,3,17594,1,NaN,50,2022-03-11 10:46:00,2022-03-11 10:47:00


In [24]:
len(PRTCPTS_BMT.STUDY_PRTCPT_ID.unique())

396

In [25]:
PRTCPTS_BMT.STUDY_PRTCPT_ID.value_counts()[0:10]

STUDY_PRTCPT_ID
CFI9I6K2    58
PF1BLPH4    58
4UDRUW84     8
FRSQKIQF     8
XGALYFG4     7
U82WP671     6
DZ5B21NY     6
ZLZ19K6V     6
B51FUYGC     6
IUPUFELI     5
Name: count, dtype: int64

In [26]:
fitbits_PF1BLPH4 = PRTCPTS_BMT[PRTCPTS_BMT.STUDY_PRTCPT_ID == 'PF1BLPH4']
fitbits_PF1BLPH4

,PRTCPT_DVC_ID,PRTCPT_DVC_TYP_ID,STUDY_PRTCPT_ID,STUDY_ID,PRTCPT_DVC_MAKE,PRTCPT_DVC_MODEL,PRTCPT_DVC_INTERNAL_ID,PRTCPT_DVC_START_DT,PRTCPT_DVC_END_DT,FITBIT_SYNC_TIME
103,20813,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-03-28 18:44:50.008,2022-03-29 09:12:34.374,2022-03-29 08:15:17
104,21053,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-04-03 20:08:42.366,2022-04-04 21:03:11.884,2022-04-04 20:55:13
105,21173,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-04-06 19:06:16.043,2022-04-07 15:51:48.944,2022-04-07 12:54:08
107,21433,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-04-16 20:15:47.206,2022-04-18 13:25:33.471,2022-04-18 12:54:02
108,21593,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-04-23 20:02:14.971,2022-04-23 20:14:21.939,2022-04-23 20:02:16
109,22013,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-05-12 20:42:38.381,2022-05-13 06:55:09.390,2022-05-13 03:51:50
110,23773,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-06-27 20:01:03.875,2022-06-28 13:32:39.849,2022-06-28 12:58:00
120,20437,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-03-01 12:09:14.376,2022-03-28 14:10:37.023,2022-03-28 12:52:44
122,21393,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-04-15 20:06:31.173,2022-04-15 21:24:01.875,2022-04-15 20:56:18
123,21853,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-05-04 20:07:44.404,2022-05-06 16:16:32.347,2022-05-06 12:53:50


Check the historical phenomena of 58 fitbits in Huron valley. 

In [27]:
fitbits_PF1BLPH4.PRTCPT_DVC_ID.iloc[0:5]

103    20813
104    21053
105    21173
107    21433
108    21593
Name: PRTCPT_DVC_ID, dtype: int64

In [28]:
for i in range(len(fitbits_PF1BLPH4.PRTCPT_DVC_ID)):
    print(fitbits_PF1BLPH4.PRTCPT_DVC_ID.iloc[i])
    print("data type of this device ID in this format is", type(str(fitbits_PF1BLPH4.PRTCPT_DVC_ID.iloc[i])))

20813
data type of this device ID in this format is <class 'str'>
21053
data type of this device ID in this format is <class 'str'>
21173
data type of this device ID in this format is <class 'str'>
21433
data type of this device ID in this format is <class 'str'>
21593
data type of this device ID in this format is <class 'str'>
22013
data type of this device ID in this format is <class 'str'>
23773
data type of this device ID in this format is <class 'str'>
20437
data type of this device ID in this format is <class 'str'>
21393
data type of this device ID in this format is <class 'str'>
21853
data type of this device ID in this format is <class 'str'>
21933
data type of this device ID in this format is <class 'str'>
21973
data type of this device ID in this format is <class 'str'>
22313
data type of this device ID in this format is <class 'str'>
22493
data type of this device ID in this format is <class 'str'>
22573
data type of this device ID in this format is <class 'str'>
22673
data

I am downloading heartrates readings from each of these fitbits for the participant PF1BLPH4, and counting how many measurements each of the devices have. 

In [29]:
df_heartrate_PF1BLPH4 = pd.DataFrame(columns=['STUDY_METRIC_ID',
'STUDY_METRIC_TYP_ID',
'PRTCPT_DVC_ID',
'PRTCPT_DVC_TYP_ID',
'STUDY_METRIC_PRTCPTN_IND',
'STUDY_METRIC_MSR_VAL',
'STUDY_METRIC_MSR_START_DT',
'STUDY_METRIC_MSR_END_DT'])

In [30]:
for i in range(len(fitbits_PF1BLPH4.PRTCPT_DVC_ID)):
    print (str(fitbits_PF1BLPH4.PRTCPT_DVC_ID.iloc[i]))
    cursor.execute(
    "select  * from roadmap.STUDY_METRIC_HEARTRATE where PRTCPT_DVC_ID = '" + str(fitbits_PF1BLPH4.PRTCPT_DVC_ID.iloc[i]) + "'"
    )
    unique_strings = cursor.fetchall()
    column_names   = [i[0] for i in cursor.description]
    df_current    = pd.DataFrame(unique_strings,columns=column_names)
    df_heartrate_PF1BLPH4  = pd.concat([df_heartrate_PF1BLPH4,df_current],ignore_index=True)
#     df_heartrate.append(df_current,ignore_index = True)
    print("device id {} is done", fitbits_PF1BLPH4.PRTCPT_DVC_ID.iloc[i])
    print ("number of heartrate measurements for this fitbit is", df_current.shape[0])

20813
device id {} is done 20813
number of heartrate measurements for this fitbit is 71685
21053
device id {} is done 21053
number of heartrate measurements for this fitbit is 88387
21173
device id {} is done 21173
number of heartrate measurements for this fitbit is 96237
21433
device id {} is done 21433
number of heartrate measurements for this fitbit is 65946
21593
device id {} is done 21593
number of heartrate measurements for this fitbit is 0
22013
device id {} is done 22013
number of heartrate measurements for this fitbit is 185559
23773
device id {} is done 23773
number of heartrate measurements for this fitbit is 146369
20437
device id {} is done 20437
number of heartrate measurements for this fitbit is 51256
21393
device id {} is done 21393
number of heartrate measurements for this fitbit is 0
21853
device id {} is done 21853
number of heartrate measurements for this fitbit is 104843
21933
device id {} is done 21933
number of heartrate measurements for this fitbit is 95308
2197

In [104]:
df_heartrate_PF1BLPH4 

,STUDY_METRIC_ID,STUDY_METRIC_TYP_ID,PRTCPT_DVC_ID,PRTCPT_DVC_TYP_ID,STUDY_METRIC_PRTCPTN_IND,STUDY_METRIC_MSR_VAL,STUDY_METRIC_MSR_START_DT,STUDY_METRIC_MSR_END_DT
0,854086848,3,20813,1,None,73,2022-03-01 12:04:00,2022-03-01 12:05:00
1,854111742,3,20813,1,None,73,2022-03-01 12:05:00,2022-03-01 12:06:00
2,854111743,3,20813,1,None,73,2022-03-01 12:05:00,2022-03-01 12:06:00
3,854111744,3,20813,1,None,77,2022-03-01 12:06:00,2022-03-01 12:07:00
4,854111745,3,20813,1,None,81,2022-03-01 12:07:00,2022-03-01 12:08:00
...,...,...,...,...,...,...,...,...
10176365,971904802,3,23873,1,None,78,2022-04-14 23:55:00,2022-04-14 23:56:00
10176366,971904803,3,23873,1,None,79,2022-04-14 23:56:00,2022-04-14 23:57:00
10176367,971904804,3,23873,1,None,80,2022-04-14 23:57:00,2022-04-14 23:58:00
10176368,971904805,3,23873,1,None,76,2022-04-14 23:58:00,2022-04-14 23:59:00


In [105]:
df_heartrate_PF1BLPH4.STUDY_METRIC_MSR_START_DT.value_counts()

STUDY_METRIC_MSR_START_DT
2022-03-04 16:25:00    139
2022-03-04 01:05:00    139
2022-03-04 01:13:00    139
2022-03-04 01:12:00    139
2022-03-04 01:11:00    139
                      ... 
2022-06-23 12:36:00      2
2022-06-23 12:37:00      2
2022-06-23 12:38:00      2
2022-06-23 12:39:00      2
2022-06-24 18:56:00      2
Name: count, Length: 155433, dtype: int64

In [114]:
abcdefg = df_heartrate_PF1BLPH4[df_heartrate_PF1BLPH4.STUDY_METRIC_MSR_START_DT=='2022-03-04 01:13:00']
abcdefg

,STUDY_METRIC_ID,STUDY_METRIC_TYP_ID,PRTCPT_DVC_ID,PRTCPT_DVC_TYP_ID,STUDY_METRIC_PRTCPTN_IND,STUDY_METRIC_MSR_VAL,STUDY_METRIC_MSR_START_DT,STUDY_METRIC_MSR_END_DT
7139,854246899,3,20813,1,None,108,2022-03-04 01:13:00,2022-03-04 01:14:00
8140,854260594,3,20813,1,None,108,2022-03-04 01:13:00,2022-03-04 01:14:00
78824,854246899,3,20813,1,None,108,2022-03-04 01:13:00,2022-03-04 01:14:00
79824,854260594,3,20813,1,None,108,2022-03-04 01:13:00,2022-03-04 01:14:00
149580,864244696,3,21053,1,None,108,2022-03-04 01:13:00,2022-03-04 01:14:00
...,...,...,...,...,...,...,...,...
9653078,968123751,3,23713,1,None,108,2022-03-04 01:13:00,2022-03-04 01:14:00
9829137,969783069,3,23794,1,None,108,2022-03-04 01:13:00,2022-03-04 01:14:00
9829636,969785746,3,23794,1,None,108,2022-03-04 01:13:00,2022-03-04 01:14:00
10036836,971628452,3,23873,1,None,108,2022-03-04 01:13:00,2022-03-04 01:14:00


In [115]:
abcdefg.STUDY_METRIC_MSR_VAL.unique()

array(['108'], dtype=object)

In [26]:
!dir

 Volume in drive C is OS-APPS
 Volume Serial Number is A6DB-57A1

 Directory of C:\Users\rajnishk\OneDrive - Michigan Medicine\Documents\Python Scripts\K24 grant

10/09/2023  10:54 AM    <DIR>          .
10/09/2023  10:54 AM    <DIR>          ..
10/03/2023  05:09 PM    <DIR>          .ipynb_checkpoints
10/09/2023  10:54 AM           186,458 BMT data heartrate.ipynb
10/03/2023  04:56 PM     6,347,199,374 BMT_heartrate.csv
10/03/2023  03:40 PM           167,142 BMT_heartrate.ipynb
10/05/2023  12:43 PM            32,777 BMT_Moods.ipynb
10/03/2023  04:51 PM        11,540,152 BMT_sleep.csv
10/03/2023  05:12 PM            54,143 BMT_sleep.ipynb
10/05/2023  12:51 PM            47,472 BMT_steps.ipynb
               7 File(s)  6,359,227,518 bytes
               3 Dir(s)  339,026,231,296 bytes free


In [31]:
PRTCPTS_BMT

,PRTCPT_DVC_ID,PRTCPT_DVC_TYP_ID,STUDY_PRTCPT_ID,STUDY_ID,PRTCPT_DVC_MAKE,PRTCPT_DVC_MODEL,PRTCPT_DVC_INTERNAL_ID,PRTCPT_DVC_START_DT,PRTCPT_DVC_END_DT,FITBIT_SYNC_TIME
0,1147,2,PUFGNCG6,201,iOS,iPhone,F38EB8B3-09E5-4FDA-9261-11DF9F85F7AD,2020-08-19 10:44:56.213,2021-02-22 12:43:49.836,NaT
1,1098,2,JZHKYTVN,201,iOS,iPhone,6DE3402C-85C8-491F-A05C-B7BA64E9B344,2020-07-24 10:01:32.081,NaT,NaT
2,1320,2,V3E5XTNB,201,iOS,iPhone,4D5F628B-A0CE-4B28-A013-D98ABB00E8EF,2020-09-15 12:59:28.546,2021-02-24 18:04:31.218,NaT
3,1332,1,V3E5XTNB,201,Fitbit,Charge HR,8V5VJT,2020-09-16 13:19:27.053,2020-09-23 11:02:11.196,2020-09-23 10:36:26
4,1744,2,ZLZ19K6V,201,iOS,iPhone,7696BC93-BBF4-40E2-8C80-9BE10373355C,2020-10-01 10:04:03.501,2020-11-11 15:50:09.558,NaT
...,...,...,...,...,...,...,...,...,...,...
977,17594,1,8GUM4J6X,323,Fitbit,Charge HR,9NGF82,2021-11-09 14:45:37.950,2022-03-12 07:17:34.828,2022-03-12 06:15:06
978,19973,2,TQR5TQ7Y,323,android,samsung SM-G781U,1643754487847,2022-02-01 17:28:09.259,2022-07-13 12:58:20.812,NaT
979,19993,2,1FUAH98J,323,android,samsung SM-G781U,1643754885693,2022-02-01 17:34:46.532,2022-07-12 16:59:27.333,NaT
980,20133,2,3BELGA8R,323,iOS,iPhone,E8140A75-2232-4816-8F20-3B5E1C18A4ED,2022-02-09 12:13:58.829,2022-07-13 12:56:38.246,NaT


In [39]:
PF1BLPH4_device_lsit = PRTCPTS_BMT[PRTCPTS_BMT.STUDY_PRTCPT_ID=='PF1BLPH4'].PRTCPT_DVC_ID.unique()
PF1BLPH4_device_lsit

array([20813, 21053, 21173, 21433, 21593, 22013, 23773, 20437, 21393,
       21853, 21933, 21973, 22313, 22493, 22573, 22673, 22773, 23173,
       21013, 21233, 21317, 21534, 21662, 21713, 20433, 21455, 21613,
       22113, 22713, 20435, 22073, 22433, 22953, 22993, 23133, 23313,
       23473, 21474, 21557, 21773, 21893, 22633, 22833, 23093, 23233,
       23413, 23533, 23593, 23633, 20893, 21133, 21813, 22173, 22353,
       23674, 23713, 23794, 23873], dtype=int64)

In [44]:
df_PRTCPT_PF1BLPH4= PRTCPTS_BMT[PRTCPTS_BMT.STUDY_PRTCPT_ID=='PF1BLPH4']
df_PRTCPT_PF1BLPH4

,PRTCPT_DVC_ID,PRTCPT_DVC_TYP_ID,STUDY_PRTCPT_ID,STUDY_ID,PRTCPT_DVC_MAKE,PRTCPT_DVC_MODEL,PRTCPT_DVC_INTERNAL_ID,PRTCPT_DVC_START_DT,PRTCPT_DVC_END_DT,FITBIT_SYNC_TIME
103,20813,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-03-28 18:44:50.008,2022-03-29 09:12:34.374,2022-03-29 08:15:17
104,21053,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-04-03 20:08:42.366,2022-04-04 21:03:11.884,2022-04-04 20:55:13
105,21173,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-04-06 19:06:16.043,2022-04-07 15:51:48.944,2022-04-07 12:54:08
107,21433,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-04-16 20:15:47.206,2022-04-18 13:25:33.471,2022-04-18 12:54:02
108,21593,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-04-23 20:02:14.971,2022-04-23 20:14:21.939,2022-04-23 20:02:16
109,22013,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-05-12 20:42:38.381,2022-05-13 06:55:09.390,2022-05-13 03:51:50
110,23773,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-06-27 20:01:03.875,2022-06-28 13:32:39.849,2022-06-28 12:58:00
120,20437,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-03-01 12:09:14.376,2022-03-28 14:10:37.023,2022-03-28 12:52:44
122,21393,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-04-15 20:06:31.173,2022-04-15 21:24:01.875,2022-04-15 20:56:18
123,21853,1,PF1BLPH4,201,Fitbit,Charge HR,4CSN82,2022-05-04 20:07:44.404,2022-05-06 16:16:32.347,2022-05-06 12:53:50


In [45]:
df_PRTCPT_CFI9I6K2= PRTCPTS_BMT[PRTCPTS_BMT.STUDY_PRTCPT_ID=='CFI9I6K2']
df_PRTCPT_CFI9I6K2

,PRTCPT_DVC_ID,PRTCPT_DVC_TYP_ID,STUDY_PRTCPT_ID,STUDY_ID,PRTCPT_DVC_MAKE,PRTCPT_DVC_MODEL,PRTCPT_DVC_INTERNAL_ID,PRTCPT_DVC_START_DT,PRTCPT_DVC_END_DT,FITBIT_SYNC_TIME
460,21353,1,CFI9I6K2,202,Fitbit,Charge HR,4CSN82,2022-04-14 11:33:10.246,2022-04-15 20:06:31.148,2022-04-15 12:54:21
461,21413,1,CFI9I6K2,202,Fitbit,Charge HR,4CSN82,2022-04-15 21:24:01.888,2022-04-16 20:15:47.179,2022-04-16 19:16:35
462,21453,1,CFI9I6K2,202,Fitbit,Charge HR,4CSN82,2022-04-18 13:25:33.483,2022-04-18 20:06:51.513,2022-04-18 20:08:11
463,22653,1,CFI9I6K2,202,Fitbit,Charge HR,4CSN82,2022-05-30 14:22:42.693,2022-05-30 21:58:15.893,2022-05-30 21:01:49
464,23394,1,CFI9I6K2,202,Fitbit,Charge HR,4CSN82,2022-06-15 23:20:14.311,2022-06-16 20:10:14.001,2022-06-16 12:57:31
467,23833,1,CFI9I6K2,202,Fitbit,Charge HR,4CSN82,2022-06-29 13:43:12.626,2022-06-30 20:14:47.244,2022-06-30 12:57:46
476,21113,1,CFI9I6K2,202,Fitbit,Charge HR,4CSN82,2022-04-04 21:03:11.908,2022-04-05 20:56:52.645,2022-04-05 20:53:35
477,21594,1,CFI9I6K2,202,Fitbit,Charge HR,4CSN82,2022-04-23 20:14:21.944,2022-04-24 19:32:54.474,2022-04-24 12:51:50
481,21793,1,CFI9I6K2,202,Fitbit,Charge HR,4CSN82,2022-05-03 09:42:12.254,2022-05-03 20:07:45.109,2022-05-03 12:52:06
482,21833,1,CFI9I6K2,202,Fitbit,Charge HR,4CSN82,2022-05-04 08:16:30.576,2022-05-04 20:07:44.384,2022-05-04 12:52:12


In [43]:
CFI9I6K2_device_lsit = PRTCPTS_BMT[PRTCPTS_BMT.STUDY_PRTCPT_ID=='CFI9I6K2'].PRTCPT_DVC_ID.unique()
CFI9I6K2_device_lsit

array([21353, 21413, 21453, 22653, 23394, 23833, 21113, 21594, 21793,
       21833, 21873, 22133, 22273, 20434, 21213, 21653, 22033, 22553,
       22693, 23153, 23253, 21153, 21313, 21553, 20793, 20833, 20913,
       21473, 21513, 21693, 22453, 22613, 22733, 20436, 21573, 21753,
       22793, 22973, 23053, 23213, 23433, 21953, 22093, 22413, 23113,
       23513, 23553, 23613, 21033, 21913, 21993, 22333, 22873, 23673,
       23693, 23753, 23793, 23913], dtype=int64)

In [46]:
CFI9I6K2_device_lsit

array([21353, 21413, 21453, 22653, 23394, 23833, 21113, 21594, 21793,
       21833, 21873, 22133, 22273, 20434, 21213, 21653, 22033, 22553,
       22693, 23153, 23253, 21153, 21313, 21553, 20793, 20833, 20913,
       21473, 21513, 21693, 22453, 22613, 22733, 20436, 21573, 21753,
       22793, 22973, 23053, 23213, 23433, 21953, 22093, 22413, 23113,
       23513, 23553, 23613, 21033, 21913, 21993, 22333, 22873, 23673,
       23693, 23753, 23793, 23913], dtype=int64)

In [47]:
PRTCPTS_BMT.STUDY_PRTCPT_ID.value_counts()[0:10]

STUDY_PRTCPT_ID
CFI9I6K2    58
PF1BLPH4    58
4UDRUW84     8
FRSQKIQF     8
XGALYFG4     7
U82WP671     6
DZ5B21NY     6
ZLZ19K6V     6
B51FUYGC     6
IUPUFELI     5
Name: count, dtype: int64

In [50]:
df_PRTCPT_LY6EY19F= PRTCPTS_BMT[PRTCPTS_BMT.STUDY_PRTCPT_ID=='LY6EY19F']
df_PRTCPT_LY6EY19F

,PRTCPT_DVC_ID,PRTCPT_DVC_TYP_ID,STUDY_PRTCPT_ID,STUDY_ID,PRTCPT_DVC_MAKE,PRTCPT_DVC_MODEL,PRTCPT_DVC_INTERNAL_ID,PRTCPT_DVC_START_DT,PRTCPT_DVC_END_DT,FITBIT_SYNC_TIME
726,1593,2,LY6EY19F,203,android,Google Pixel 2 XL,1601388150941,2020-09-29 10:02:31.095,2020-12-11 18:29:27.190,NaT
727,1725,1,LY6EY19F,203,Fitbit,Charge HR,2C6MN5,2020-09-30 12:58:27.866,NaT,2022-06-05 02:19:15
741,9798,2,LY6EY19F,203,iOS,iPhone,E57BE8EC-201F-4891-A94F-80D35034D2DD,2020-12-11 18:31:11.094,2021-01-28 18:28:35.321,NaT


In [51]:
df_PRTCPT_FRSQKIQF= PRTCPTS_BMT[PRTCPTS_BMT.STUDY_PRTCPT_ID=='FRSQKIQF']
df_PRTCPT_FRSQKIQF

,PRTCPT_DVC_ID,PRTCPT_DVC_TYP_ID,STUDY_PRTCPT_ID,STUDY_ID,PRTCPT_DVC_MAKE,PRTCPT_DVC_MODEL,PRTCPT_DVC_INTERNAL_ID,PRTCPT_DVC_START_DT,PRTCPT_DVC_END_DT,FITBIT_SYNC_TIME
263,1596,2,FRSQKIQF,202,iOS,iPhone,8ACBAA84-4778-4006-94DD-AC506CD7C12D,2020-09-29 10:08:46.748,2020-12-11 18:28:17.544,NaT
264,1724,1,FRSQKIQF,202,Fitbit,Charge HR,8VWLMZ,2020-09-30 12:55:37.287,2020-10-23 11:03:42.554,2020-10-09 16:53:47
271,4153,1,FRSQKIQF,202,Fitbit,Charge HR,8VWLMZ,2020-10-23 13:52:13.485,2020-11-25 16:52:40.979,2020-11-24 22:16:51
282,9072,1,FRSQKIQF,202,Fitbit,Charge HR,8VLT7W,2020-11-26 13:16:09.629,2020-12-14 11:08:27.234,2020-12-10 22:58:26
293,9799,2,FRSQKIQF,202,iOS,iPhone,CD01858E-C8F5-4658-85F4-DDAAD3C44A82,2020-12-11 18:36:17.220,2020-12-11 18:39:16.771,NaT
294,9861,2,FRSQKIQF,202,iOS,iPhone,2490661A-F3DD-42F1-A2E9-DB2E3CBDBB33,2020-12-14 10:58:08.842,2021-01-28 18:29:33.176,NaT
301,9864,1,FRSQKIQF,202,Fitbit,Charge HR,8VLT7W,2020-12-14 11:11:09.136,2020-12-14 11:21:04.495,2020-12-14 11:11:10
310,9866,1,FRSQKIQF,202,Fitbit,Charge HR,8VLT7W,2020-12-14 11:21:33.018,NaT,2023-10-06 07:00:19
